<a href="https://colab.research.google.com/github/jvitordeoliveira96/UFRJ_courses/blob/main/ALA_ICP115/Tutorials/Plotly_3D_planes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from typing import Tuple, Iterable
import plotly.offline as py
from plotly.offline import plot, iplot
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'colab'
import sympy as sym


In [ ]:
#@title Funções que realizam eliminação Gaussiana (sem subs reversa)

def permutation_check(A_aug, l, c):
  for k in range(l+1, A_aug.shape[0]):
    if (A_aug[k, c] != 0):
      return k
    
  return -1


def gaussian_elimination(A, b = None, show_steps = True, save_iters = False):
  """ Computes Gaussian elimination (row echoleon form), showing step by step. 
  Sympy Matrix, Sympy Matrix -> Sympy Matrix """

  if (b == None):
    A_aug = A.copy()
  else:
    A_aug = A.col_insert(A.shape[1], b)

  if (save_iters):
    A_aug_iters = []
    A_aug_iters.append(A_aug.copy())
  
  if (show_steps):
    print('\033[1mInício\033[0m')
    print('--------------------------------------')
    display(A_aug)
    print('\n')
    print('--------------------------------------')

  el = 0
  for j in range(0, A.shape[1]):
    pivot_in_column = False

    if (A_aug[el,j] == 0):
      if (show_steps):
        print('Atenção: pivô zero em A[{0}, {1}]!'. format(el, j) )
        display(A_aug)
      
      l = permutation_check(A_aug, el, j)
      if ( l !=-1 ):
        A_aug[l, :], A_aug[el, :] = A_aug[el, :], A_aug[l, :]
        if (show_steps):
          print('Permutação L{0} <-> L{1}'.format(l, el))
          display(A_aug)
        pivot_in_column = True
        if (save_iters):
          A_aug_iters.append(A_aug.copy())

    else:
      pivot_in_column = True
    
    if ( pivot_in_column ):
      for i in range(el+1, A_aug.shape[0]):
        c = - A_aug[i, j] / A_aug[el, j] 
        A_aug[i, :] = A_aug[i, :] + c * A_aug[el, :]
        if (show_steps):
          print('L{0} <- L{0} + ({2}) * L{1}'.format(i, el, c))
          display(A_aug)
          print('\n')
        if (save_iters):
          A_aug_iters.append(A_aug.copy())
      
      el = (el + 1) if (el + 1 < A_aug.shape[0] ) else el 
      if (show_steps):  
        print('--------------------------------------')

  if (show_steps):
    print('\033[1mFim\033[0m')
  
  if (save_iters):
    return A_aug_iters
  else:
    return A_aug





In [ ]:
#@title Funcoes para criacao de planos (a partir do vetor normal) e retas no R^3
def Plane(normal: Tuple[int, int, int],
              d: int,
              colorScaleName: str, opacity = 1, values = range(-80, 80)):
    """
        :param fig: figure to plot on
        :param colorScaleName: choose from <https://plotly.com/javascript/colorscales/>
    """

    if (normal[2]!=0):
      # x, y, z
      x, y = np.meshgrid(values, values)
      z = (-normal[0] * x - normal[1] * y - d) * 1. / normal[2]
    else:
      # x, z
      x, z = np.meshgrid(values, values)
      y = (-normal[0] * x - normal[2] * z - d) * 1. / (normal[1] + 1e-3)
    

    # draw plane
    surface = go.Surface(x=x, y=y, z=z, colorscale=colorScaleName, showscale=False, opacity = opacity)
    return surface


def vector3D(vec: Iterable, colorName: str, 
             origin = [0,0,0]):
  """
    Function to create 3D array, starting from the origin as default. 
    based on https://stackoverflow.com/questions/43164909/plotlypython-how-to-plot-arrows-in-3d 
    Add this list elements to data in order to plot.
  """ 
  return [
          {
        "x": [origin[0], vec[0]],
        "y": [origin[1], vec[1]],
        "z": [origin[2], vec[2]],
        "mode": "lines",
        "type": "scatter3d",
        "line": {
            "color": colorName,
            "width": 3
        }
    },
    {
        "type": "cone",
        "x": [vec[0]],
        "y": [vec[1]],
        "z": [vec[2]],
        "u": [0.3*(vec[0])],
        "v": [0.3*(vec[1])],
        "w": [0.3*(vec[2])],
        "anchor": "tip", # make cone tip be at endpoint
        "hoverinfo": "none",
        "colorscale": [[0, colorName], [1, colorName]], # color all cones colorName
        "showscale": False,
    }
  ]

In [ ]:
A = sym.Matrix([[1, 1, 3], [-1, -2, 1], [3, 5, 1]])

In [ ]:
gaussian_elimination(A)
# ou 
#b = sym.Matrix([0,0,0])
#gaussian_elimination(A, b)

Início
--------------------------------------


Matrix([
[ 1,  1, 3],
[-1, -2, 1],
[ 3,  5, 1]])



--------------------------------------
L1 <- L1 + (1) * L0


Matrix([
[1,  1, 3],
[0, -1, 4],
[3,  5, 1]])



L2 <- L2 + (-3) * L0


...

Matrix([
[1,  1, 3],
[0, -1, 4],
[0,  0, 0]])



--------------------------------------
Atenção: pivô zero em A[2, 2]!


Matrix([
[1,  1, 3],
[0, -1, 4],
[0,  0, 0]])

Fim


Matrix([
[1,  1, 3],
[0, -1, 4],
[0,  0, 0]])

## Visão colunas

In [ ]:
origin = np.array((0,0,0))
v = np.array(A[:, 0]).astype(float).flatten()
w = np.array(A[:, 1]).astype(float).flatten()
z = np.array(A[:, 2]).astype(float).flatten()

s3 = Plane(np.cross(v, w), 0, "Greens", opacity = 0.5)

vdraw, wdraw, zdraw = vector3D(v, "blue"), vector3D(w, "red"), vector3D(z, "purple")

data = vdraw + wdraw + zdraw + [s3]

rangexy = [-5.1,5.1]
rangez = [-5.1,5.1]
layout = go.Layout(scene = dict(
                    xaxis = dict(nticks=4, range=rangexy),
                     yaxis = dict(nticks=4, range=rangexy,),
                     zaxis = dict(nticks=4, range=rangez,), 
                     aspectratio = {"x": 1.3, "y":1.3, "z":1}, 
                     aspectmode = "manual"),
                   margin = dict( l = 0,
                                  r = 0,
                                  b = 0,
                                  t = 0)
                  )
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Visão das linhas

In [ ]:
origin = np.array((0,0,0))
n1 = np.array(A[0, :]).astype(float).flatten()
n2 = np.array(A[1, :]).astype(float).flatten()
n3 = np.array(A[2, :]).astype(float).flatten()

s1 = Plane(n1, 0, "Hot", opacity = 1)
s2 = Plane(n2, 0, "Greys", opacity = 1)
s3 = Plane(n3, 0, "Greens", opacity = 1)

data = [s1, s2, s3]

rangexy = [-8.1,8.1]
rangez = rangexy
layout = go.Layout(scene = dict(
                    xaxis = dict(nticks=4, range=rangexy),
                     yaxis = dict(nticks=4, range=rangexy,),
                     zaxis = dict(nticks=4, range=rangez,), 
                     aspectratio = {"x": 1.3, "y":1.3, "z":1}, 
                     aspectmode = "manual"),
                   margin = dict( l = 0,
                                  r = 0,
                                  b = 0,
                                  t = 0)
                  )
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## Exemplo slide

In [ ]:
A = sym.Matrix([[1,1],[0,0],[-1,0]])
B = sym.Matrix([[0,0],[0,1],[-1,0]])

In [ ]:
origin = np.array((0,0,0))
v1 = np.array(A[:, 0]).astype(float).flatten()
v2 = np.array(A[:, 1]).astype(float).flatten()
t1 = np.array(B[:, 0]).astype(float).flatten()
t2 = np.array(B[:, 1]).astype(float).flatten()

s2 = Plane(np.cross(t1, t2), 0, "Hot", opacity = 0.5)
s3 = Plane(np.cross(v1, v2), 0, "Greens", opacity = 0.5)

vdraw, v2draw, t1draw, t2draw = vector3D(v1, "blue"), vector3D(v2, "red"), vector3D(t1, "purple"), vector3D(t2, "yellow")

data = vdraw + v2draw + t1draw + t2draw + [s2] + [s3]

rangexy = [-2.2,2.2]
rangez = [-2.2,2.2]
layout = go.Layout(scene = dict(
                    xaxis = dict(nticks=4, range=rangexy),
                     yaxis = dict(nticks=4, range=rangexy,),
                     zaxis = dict(nticks=4, range=rangez,), 
                     aspectratio = {"x": 1.3, "y":1.3, "z":1}, 
                     aspectmode = "manual"),
                   margin = dict( l = 0,
                                  r = 0,
                                  b = 0,
                                  t = 0)
                  )
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)